In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from analysts_tools.redash_methods import *
    #Frubana libraries
from procurement_lib import redash, dw, send_message, send_slack_notification, GoogleSheet, logging

In [2]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros
"""Todays date should be the first day of the next month which is going to begin with SD"""

city_code = 'BAQ'

# Set today's date to the first day of the next month
today = datetime.today()
first_day_next_month = (today.replace(day=1) + timedelta(days=32)).replace(day=1)

# Calculate non_buyers_date (todays date - 1 month)
non_buyers_date = (first_day_next_month - timedelta(days=1)).replace(day=1)
customer_date_desde = (first_day_next_month - timedelta(days=1)).replace(day=1)

# Calculate sku_date_desde (todays date - 2 months)
sku_date_desde = (first_day_next_month - timedelta(days=60)).replace(day=1)

# Convert the dates to strings in the desired format
todays_date = first_day_next_month.strftime('%Y-%m-%d')
non_buyers_date = non_buyers_date.strftime('%Y-%m-%d')
customer_date_desde = customer_date_desde.strftime('%Y-%m-%d')
sku_date_desde = sku_date_desde.strftime('%Y-%m-%d')

print(todays_date) # Output: e.g. 2023-04-01
print(non_buyers_date) # Output: e.g. 2023-03-01
print(sku_date_desde) # Output: e.g. 2023-02-01

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

2023-07-01
2023-06-01
2023-05-01


In [3]:
query="""
With microsegment as (

SELECT DISTINCT c.customer_id AS user_id,
                c.site_type AS region_code_ms,
                
                (case 
                    -- Colombia
                    when m.microsegment_id not in (1,3,4,28,5,6,7,8,2) and c.site_type = 'BOG' then 'Sin Microsegmento'
                    when m.microsegment_id not in (1,3,4,28,5,11,10,9,2,12) and c.site_type = 'BAQ' then 'Sin Microsegmento'
                    when m.microsegment_id not in (3,4,28,1,24,11,5,2,27) and c.site_type = 'MDE' then 'Sin Microsegmento'
    
                    -- Mexico
                    when m.microsegment_id not in (1,13,14,30,15,16,29,17,6,7,2) and c.site_type = 'CMX' then 'Sin Microsegmento'
                    when m.microsegment_id not in (1, 13,14,30, 15,16,29, 6, 33, 2, 31) and c.site_type = 'GDL' then 'Sin Microsegmento'
                    when m.microsegment_id not in (13,14,30, 15,16,29, 32,2,6,33) and c.site_type = 'PBC' then 'Sin Microsegmento'
                    
                    -- Brasil
                    when m.microsegment_id not in (21,34,38,37,35,36,39,26) and c.site_type in ('SPO','BHZ','CWB','VCP') then 'Sin Microsegmento'
                        
                else m.description end) as microsegment_ok,
                
                m.microsegment_id,
                mo.name as ms_method
                
                
FROM postgres_main_federate."growth_microsegmentation.customer_microsegment" AS cm
INNER JOIN postgres_main_federate."growth_microsegmentation.microsegment" m ON cm.microsegment_id = m.id
INNER JOIN postgres_main_federate."growth_microsegmentation.customer" c ON cm.customer_id = c.id
INNER JOIN postgres_main_federate."growth_microsegmentation.models" mo ON mo.id = cm.model_type_id


)

select m.user_id,
m.region_code_ms,
--m.microsegment_id,

(case 
        when m.microsegment_ok in ('Corrientazo c/desayuno','Corrientazo s/desayuno','Tienda/Restaurante') then 'Corrientazo' 
        when m.microsegment_ok in ('Fonda c/desayuno','Fonda s/desayuno') then 'Fonda'
        when m.microsegment_ok in ('Taquería en calle','Taquería en local') then 'Taquería'

else m.microsegment_ok end) as microsegment,

m.ms_method

from microsegment m
where m.microsegment_ok != 'Sin Microsegmento'
"""

segmentos_fulll = read_connection_data_warehouse.runQuery(query)

segmentos_fulll.columns = ['customer_id', 'city', 'microsegment','NA']
segmentos_fulll = segmentos_fulll[['customer_id', 'city', 'microsegment']]

### Loading COSTUMERS INFO

In [4]:
users_eliminar_kam = get_fresh_query_result("https://internal-redash.federate.frubana.com/",150812,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
users_eliminar = users_eliminar_kam.loc[users_eliminar_kam.registered_city == city_code]['customer_id'].unique().tolist()

200


### Loading FORCED_SKUS // Cleaning SEGMENT_NAMES

### Loading DATA_VENTAS // Removing SKUs /\ CUSTOMERS_ID

In [5]:
#//Remove skus 
#It is used because some SKUs can get in trouble with supplier for the SD or to exclude them for another activity   
skus_remove = GoogleSheet("1Z0T_c36oRHmOHIgEG8_ANcwFykaHgXGAizYAYCKCqtY")
skus_remove = skus_remove.get_as_dataframe('SKUS')

skus_remove = skus_remove['sku_id'].unique().astype(int).tolist()

In [6]:
query = """
SELECT DISTINCT
    split_part(bs.upc, '-', 1) as region,
    bs.sku_id

FROM postgres_broadleaf_federate."broadleaf.blc_sku"            bs 
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_product"   bp ON bp.product_id = bs.addl_product_id

WHERE bs.archived = 'N'
    AND bs.sndbx_id is NULL
    AND bp.archived = 'N'
    AND bs.catalog_disc < 0
    AND ((bp.archived = 'N' AND bp.sndbx_id is NULL) OR (bp.archived is NULL))
"""
frida_products = read_connection_data_warehouse.runQuery(query)

In [7]:
good_skus = frida_products.sku_id.unique()

In [8]:
#Loading SELL'S DATA from a function
data_ventas = data_ventas_query(ciudad=city_code, tipo_cambio=t_cambio, desde=customer_date_desde, hasta = todays_date, tipo_negocio=1)

In [9]:
data_ventas = data_ventas[(~data_ventas['sku_id'].isin(skus_remove)) & (data_ventas['sku_id'].isin(good_skus))]

In [10]:
segmentos_full = segmentos_fulll[segmentos_fulll.city == city_code].copy()

In [11]:
#//customers for offers */(has further uses)*/
microsegmentacion_df = segmentos_full[(~segmentos_full['customer_id'].isin(users_eliminar))]
data_ventas = data_ventas[(~data_ventas['customer_id'].isin(users_eliminar))]

### Create the MS "Other" for BAU 

In [12]:
data_full = data_ventas.merge(microsegmentacion_df[['customer_id','microsegment']], how = 'left', on = 'customer_id')

In [13]:
data_full['microsegment'] = data_full['microsegment'].fillna('Other')

In [14]:
data_full[['customer_id','microsegment']].groupby(['microsegment']).nunique().reset_index()

,microsegment,customer_id
0,Comida de mar y Colombiana,14
1,Comida especializada,97
2,Comida rápida,482
3,Corrientazo,1373
4,Fritos,437
5,Other,264
6,Panadería/Pastelería/Cafetería,622
7,Parrilla/Asadero,122


In [15]:
segments_validation = data_full[['customer_id','microsegment']].groupby(['microsegment']).nunique().reset_index()

In [16]:
for ms in segments_validation[segments_validation.customer_id < 180]['microsegment'].unique():
    data_full['microsegment'] = data_full['microsegment'].str.replace(ms,'Other')

In [17]:
data_full[['customer_id','microsegment']].groupby(['microsegment']).nunique().reset_index()

,microsegment,customer_id
0,Comida rápida,482
1,Corrientazo,1373
2,Fritos,437
3,Other,497
4,Panadería/Pastelería/Cafetería,622


In [18]:
#Get the same variable to do the process job
data_ventas = data_full

### PARAMETERS // SOME FUNCTIONS - GET READY TO PROCESSING

In [19]:
def ms_data(city_code):
  
    paramss_csv=GoogleSheet("1k0ckgkLddxc5lY8NUVGCkvDLTuulLaJm-N535oMNZNw")
    city_paramss = paramss_csv.get_as_dataframe(city_code)
    
    parametros = {}
    parametros[city_code] = {}
    
    for i,ms in enumerate(city_paramss.iloc[:,4:].columns):
        parametros[city_code][ms] = [(value[1],value[2],value[3],value[4+i]) for value in city_paramss.values if value[4+i]>0]
        
    return parametros

In [20]:
parametros = ms_data(city_code)

In [21]:
def normalize_text(raw_text):
    """
    Removes common accent characters and normalize text.
    """
    raw_text = raw_text.replace(" ", "")
    raw_text = raw_text.replace("�", "")
    raw_text = raw_text.replace("/", "")
    raw_text = re.sub(r'\s*', '', raw_text)
    raw_text = re.sub(r"[àáâãäå]", 'a', raw_text)
    raw_text = re.sub(r"[èéêë]", 'e', raw_text)
    raw_text = re.sub(r"[ìíîï]", 'i', raw_text) 
    raw_text = re.sub(r"[òóôõö]", 'o', raw_text)
    raw_text = re.sub(r"[ùúûü]", 'u', raw_text)
    raw_text = re.sub(r"[ýÿ]", 'y', raw_text)
    raw_text = re.sub(r"[ß]", 'ss', raw_text)
    raw_text = re.sub(r"[ñ]", 'n', raw_text)
    raw_text = re.sub(r"[ç]", 'c', raw_text)
    raw_text = raw_text.lower()
    return raw_text

In [22]:
#Sacar de aca los microsegmentos
msg = data_full.microsegment.unique()
msg

array(['Comida rápida', 'Corrientazo', 'Panadería/Pastelería/Cafetería',
       'Fritos', 'Other'], dtype=object)

### END

# Aca Empieza la Experimentacion

In [23]:
from sklearn.model_selection import train_test_split 

In [24]:
#//Variables to store all the segments and offers of the city
global_offer_csv = pd.DataFrame()
global_segment_list = [] #//for procesing

In [25]:
#//----------------------------------------------------------------------------
#//Big loop for creating each city segments/offers files
aux_df = pd.DataFrame()
aux_df['aux']=6000*[0]

for microsegment in msg:
    print('//-------------------------------------------------------------------------------')
    print(microsegment)

    #//Customers per microsegment
    customers_ms = data_ventas[(data_ventas['microsegment'] == microsegment)]['customer_id'].unique()
    #//Customers to give them the skus
    customers_for_skus_ms =  data_ventas[['microsegment','customer_id']][(data_ventas.microsegment == microsegment) & (~data_ventas.customer_id.isin(users_eliminar))]['customer_id'].unique()
    
    #----------------------------------------------------------- START OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    
    
    #//----------------------------------------------------------------------------------------------------------------------------
    ms_name = normalize_text(microsegment)+"_PILOTO" #NORMALIZE TEXT FORMAT
    #//----------------------------------------------------------------------------------------------------------------------------
    
    #// Separation of types of sales (non_buyers/skus for offers)
    #//////////////////////////////// THIS IS FOR CREATE EXPERIMENTS //////////////////////////////////////////////
    rand=1
    customer_ms_2, customer_ms_1 = train_test_split(customers_ms, test_size = 0.80, random_state=rand)
    
    while abs(1-(data_ventas[data_ventas['customer_id'].isin(customer_ms_2)]['gmv_usd'].mean()/data_ventas[data_ventas['customer_id'].isin(customer_ms_1)]['gmv_usd'].mean())) > 0.05:
        #print(data_ventas[data_ventas['customer_id'].isin(customer_ms_1)]['gmv_usd'].mean(),data_ventas[data_ventas['customer_id'].isin(customer_ms_2)]['gmv_usd'].mean(),abs(1-(data_ventas[data_ventas['customer_id'].isin(customer_ms_2)]['gmv_usd'].mean()/data_ventas[data_ventas['customer_id'].isin(customer_ms_1)]['gmv_usd'].mean())))
        rand+=1
        customer_ms_2, customer_ms_1 = train_test_split(customers_ms, test_size = 0.80, random_state = rand)
        
    #//////////////////////////////// THIS IS FOR CREATE EXPERIMENTS //////////////////////////////////////////////
    
    data_buyers = data_ventas[data_ventas['customer_id'].isin(customer_ms_1)].copy() #le quite la s!!
    
    print(data_buyers.customer_id.nunique())
    data_skus = data_ventas[data_ventas['customer_id'].isin(customers_for_skus_ms)].copy()
    
    subcats = list(map(lambda x: x[0], parametros[city_code][microsegment]))
    max_uses = list(map(lambda x: int(x[1]), parametros[city_code][microsegment]))
    max_order = list(map(lambda x: int(x[2]), parametros[city_code][microsegment]))
    discount = list(map(lambda x: int(x[3]), parametros[city_code][microsegment]))

    #----------------------------------------------------------- END OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    
    
    #----------------------------------------------------------- START OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    a, b, c = proceso_para_csv(data_ventas=data_buyers, todays_date=todays_date, 
            non_buyers_date=non_buyers_date, city=city_code, list_to_filter=subcats, offer_discount=discount, offer_max_uses=max_uses, offer_max_order=max_order, level='subcat', additional_words=ms_name,
                        limite=0.75, data_ventas_skus = data_skus)

    print(b.nunique())
    #//appending to global data frame
    global_offer_csv = global_offer_csv.append(a)
    
    #//Data formating
    lst = []
    name = 'export_data/MS/nb_'+city_code+'_'+ms_name+'.txt'
    for seg in b.columns:
        id_list = b[seg].dropna().astype(int)
        d = {'name':seg, 'customersIds': list(id_list)}
        aux_df.loc[range(len(list(id_list))),seg] = list(id_list)

        lst.append(d)
        
    #//appending to global list    
        global_segment_list.append(d)
    
    print(f'Done {city_code} {ms_name}')

#----------------------------------------------------------- END OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------


#----------------------------------------------------------- START OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    
    
    #//----------------------------------------------------------------------------------------------------------------------------
    ms_name = normalize_text(microsegment)+"_CONTROL" #NORMALIZE TEXT FORMAT
    #//----------------------------------------------------------------------------------------------------------------------------
    
    #// Separation of types of sales (non_buyers/skus for offers)
    #//////////////////////////////// THIS IS FOR CREATE EXPERIMENTS //////////////////////////////////////////////
    data_buyers = data_ventas[data_ventas['customer_id'].isin(customer_ms_2)].copy() #le quite la s!!
    #//////////////////////////////// THIS IS FOR CREATE EXPERIMENTS //////////////////////////////////////////////
    
    print('//-------------------------------------------------------------------------------')
    print(f'{microsegment} Experimento50%')
    print(data_buyers.customer_id.nunique())
    data_skus = data_ventas[data_ventas['customer_id'].isin(customers_for_skus_ms)].copy()

    #----------------------------------------------------------- END OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    
    
    #----------------------------------------------------------- START OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    e, f, g = proceso_para_csv(data_ventas=data_buyers, todays_date=todays_date, 
                non_buyers_date=non_buyers_date, city=city_code, list_to_filter=subcats, offer_discount=discount, offer_max_uses=max_uses, offer_max_order=max_order, level='subcat', additional_words=ms_name,
                          limite=0.75, data_ventas_skus = data_skus)
      
    print(f.nunique())
    #//appending to global data frame
    global_offer_csv = global_offer_csv.append(e)
    
    #//Data formating
    lst = []
    name = 'export_data/MS/nb_'+city_code+'_'+ms_name+'.txt'
    for seg in f.columns:
        id_list = f[seg].dropna().astype(int)
        h = {'name':seg, 'customersIds': list(id_list)}
        aux_df.loc[range(len(list(id_list))),seg] = list(id_list)
        
        lst.append(h)
        
        
    #//appending to global list    
        global_segment_list.append(h)
    
    print(f'Done {city_code} {ms_name}')
    #----------------------------------------------------------- END OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    
#//End of big loop and start of data bundle 

//-------------------------------------------------------------------------------
Comida rápida
386
Uso de data externa
20230701_BAQ_comidarapida_piloto_NB_aceitesygrasas_W26                 161
20230701_BAQ_comidarapida_piloto_NB_arroz_W26                          301
20230701_BAQ_comidarapida_piloto_NB_azucaryendulzantes_W26             312
20230701_BAQ_comidarapida_piloto_NB_harinasymezclas_W26                357
20230701_BAQ_comidarapida_piloto_NB_pasta_W26                          370
20230701_BAQ_comidarapida_piloto_NB_salsas_W26                         341
20230701_BAQ_comidarapida_piloto_NB_desinfectantesysanitizantes_W26    309
20230701_BAQ_comidarapida_piloto_NB_detergentejabonylavalozas_W26      278
20230701_BAQ_comidarapida_piloto_NB_papelytoallas_W26                  215
20230701_BAQ_comidarapida_piloto_NB_aguas_W26                          370
20230701_BAQ_comidarapida_piloto_NB_cervezasylicores_W26               370
20230701_BAQ_comidarapida_piloto_NB_gaseosas_W26       

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


Uso de data externa
20230701_BAQ_corrientazo_control_NB_aceitesygrasas_W26                 139
20230701_BAQ_corrientazo_control_NB_arroz_W26                          197
20230701_BAQ_corrientazo_control_NB_azucaryendulzantes_W26             193
20230701_BAQ_corrientazo_control_NB_granos_W26                         222
20230701_BAQ_corrientazo_control_NB_harinasymezclas_W26                252
20230701_BAQ_corrientazo_control_NB_pasta_W26                          257
20230701_BAQ_corrientazo_control_NB_salsas_W26                         238
20230701_BAQ_corrientazo_control_NB_desinfectantesysanitizantes_W26    223
20230701_BAQ_corrientazo_control_NB_detergentejabonylavalozas_W26      181
20230701_BAQ_corrientazo_control_NB_papelytoallas_W26                  140
20230701_BAQ_corrientazo_control_NB_aguas_W26                          268
20230701_BAQ_corrientazo_control_NB_gaseosas_W26                       225
20230701_BAQ_corrientazo_control_NB_jugos_W26                          233
20230

## Aca creo el archivo para R&D

In [26]:
# customers =  pd.DataFrame()
# for column,i in enumerate(aux_df.columns):
#     #if ('verduras' in i or 'frutas' in i or 'arroz' in i or 'aceitesygrasas' in i or 'harinasymezclas' in i or 'azucaryendulzantes' in i or 'pollo' in i or 'huevos' in i) and not 'frutasjugo' in i:
#     #if ('verduras' in i or 'frutas' in i) and not 'frutasjugo' in i:
#     customers[i] = aux_df[i]

In [27]:
# customers.iloc[:6000].to_csv(f'SD/export_data/MS/csv_{city_code}_CLIENTS_PRUEBA_R&D.csv', index = False)

# with open(f'SD/export_data/MS/nb_{city_code}_All_MS.txt', 'w') as outfile:
#        json.dump(global_segment_list, outfile)

In [28]:
# aux_seg = pd.read_csv(f"prueba_tuberculos_BAQ.csv")
# aux_seg["customer_source_id"] = aux_seg.customer_source_id.astype(int)

# aux_seg["final_group2"] = ["piloto" if x == "Super Descuentos" else "control" for x in aux_seg.grupos]
# aux_seg['offer_name'] = f"20230301_{city_code}_" + aux_seg.ms + "_" + aux_seg.final_group2 + "_NB_" + aux_seg.subcategory + "_W9"

# aux_seg2 = aux_seg.groupby(by = ["offer_name"])["customer_source_id"].apply(list).reset_index()

In [29]:
# lst = []
# for x in aux_seg2.values:
#     h = {'name':x[0], 'customersIds': list(x[1])}
#     lst.append(h)
    
# for x in lst:
#     print(x['name'])
#     #break
#     for i,value in enumerate(global_segment_list):
#         if value['name'] == x['name']:
#             global_segment_list[i]['customersIds'] = x['customersIds']
#             print(i)

In [30]:
aux_seg = pd.read_csv(f"csv_{city_code}_CLIENTS_FINAL_JULIO_R&D.csv")
aux_seg["customer_source_id"] = aux_seg.customer_source_id.astype(int)

aux_seg["final_group2"] = ["piloto" if x == "Treatment" else "control" for x in aux_seg.final_group]
aux_seg['offer_name'] = f"20230701_{city_code}_" + aux_seg.ms + "_" + aux_seg.final_group2 + "_NB_" + aux_seg.subcategory + "_W26"

aux_seg["quartile"] = ["Q0" if x == "-" else x for x in aux_seg.quartile.values]
aux_seg["quartile_name"] = f"20230701_{city_code}_" + aux_seg.subcategory + "_" + aux_seg.quartile + "_" + aux_seg.case

aux_seg2 = aux_seg.groupby(by = ["offer_name"])["customer_source_id"].apply(list).reset_index()
aux_seg3 = aux_seg.groupby(by = ["quartile_name"])["customer_source_id"].apply(list).reset_index()

In [31]:
aux_seg.head()

,subcategory,customer_source_id,ms,previous_group,case,quartile,random,final_group,final_group2,offer_name,quartile_name
0,aceitesygrasas,260795315,corrientazo,Control,positivity,Q4,0,Treatment,piloto,20230701_BAQ_corrientazo_piloto_NB_aceitesygra...,20230701_BAQ_aceitesygrasas_Q4_positivity
1,aceitesygrasas,237802737,comidarapida,Treatment,positivity,Q4,0,Treatment,piloto,20230701_BAQ_comidarapida_piloto_NB_aceitesygr...,20230701_BAQ_aceitesygrasas_Q4_positivity
2,aceitesygrasas,45899933,other,Control,positivity,Q4,0,Treatment,piloto,20230701_BAQ_other_piloto_NB_aceitesygrasas_W26,20230701_BAQ_aceitesygrasas_Q4_positivity
3,aceitesygrasas,76812821,panaderiapasteleriacafeteria,Treatment,positivity,Q4,0,Treatment,piloto,20230701_BAQ_panaderiapasteleriacafeteria_pilo...,20230701_BAQ_aceitesygrasas_Q4_positivity
4,aceitesygrasas,59820190,panaderiapasteleriacafeteria,Treatment,positivity,Q4,0,Treatment,piloto,20230701_BAQ_panaderiapasteleriacafeteria_pilo...,20230701_BAQ_aceitesygrasas_Q4_positivity


In [32]:
lst = []
for x in aux_seg2.values:
    h = {'name':x[0], 'customersIds': list(x[1])}
    lst.append(h)

global_segment_list = lst

In [33]:
lst2 = []
for x in aux_seg3.values:
    h = {'name':x[0], 'customersIds': list(x[1])}
    lst2.append(h)

# Aca tengo que volver a cargar el archivo de arriba y dejarlo como lista para traer lo otro

In [34]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,20230701_BAQ_comidarapida_piloto_NB_aceitesygr...,comidarapida_piloto_NB_aceitesygrasas,6,true,01-07-2023,31-07-2023,7,1,None,632389#624989#222614
1,20230701_BAQ_comidarapida_piloto_NB_arroz_W26,comidarapida_piloto_NB_arroz,4,true,01-07-2023,31-07-2023,7,1,None,554034#281192
2,20230701_BAQ_comidarapida_piloto_NB_azucaryend...,comidarapida_piloto_NB_azucaryendulzantes,4,true,01-07-2023,31-07-2023,5,1,None,83616#600127
3,20230701_BAQ_comidarapida_piloto_NB_harinasyme...,comidarapida_piloto_NB_harinasymezclas,4,true,01-07-2023,31-07-2023,6,1,None,30861#257373#602333
4,20230701_BAQ_comidarapida_piloto_NB_pasta_W26,comidarapida_piloto_NB_pasta,12,true,01-07-2023,31-07-2023,0,1,None,257365#257363


In [35]:
#len_seg =int(len(global_segment_list)/10)
#len_seg_list = [global_segment_list[:len_seg],global_segment_list[len_seg:len_seg*2],global_segment_list[len_seg*2:len_seg*3],global_segment_list[len_seg*3:len_seg*4],global_segment_list[len_seg*4:len_seg*5],global_segment_list[len_seg*5:len_seg*6],global_segment_list[len_seg*6:len_seg*7],global_segment_list[len_seg*7:len_seg*8],global_segment_list[len_seg*8:len_seg*9],global_segment_list[len_seg*9:]]

## Crear segments offers y segments types

In [36]:
import requests
import json

def create_segments(segment_list):
    
    url = f"https://{city_code}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

    payload = json.dumps(segment_list)


    headers = {
    'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
    'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.status_code)
    sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]
    time.sleep(40)
    return sgt

In [37]:
len_seg =int(len(global_segment_list)/10)
len_seg_list = [global_segment_list[:len_seg],global_segment_list[len_seg:len_seg*2],global_segment_list[len_seg*2:len_seg*3],global_segment_list[len_seg*3:len_seg*4],global_segment_list[len_seg*4:len_seg*5],global_segment_list[len_seg*5:len_seg*6],global_segment_list[len_seg*6:len_seg*7],global_segment_list[len_seg*7:len_seg*8],global_segment_list[len_seg*8:len_seg*9],global_segment_list[len_seg*9:]]

In [38]:
#offers
sgt=[]

for i in len_seg_list:
    sgt.append(create_segments(i))
    
sgt_end =[item for lista in sgt for item in lista]

200
200
200
200
200
200
200
200
200
200


In [39]:
len_seg2 =int(len(lst2)/10)
len_seg_list2 = [lst2[:len_seg2],lst2[len_seg2:len_seg2*2],lst2[len_seg2*2:len_seg2*3],lst2[len_seg2*3:len_seg2*4],lst2[len_seg2*4:len_seg2*5],lst2[len_seg2*5:len_seg2*6],lst2[len_seg2*6:len_seg2*7],lst2[len_seg2*7:len_seg2*8],lst2[len_seg2*8:len_seg2*9],lst2[len_seg2*9:]]

In [40]:
#types
sgt2=[]

for i in len_seg_list2:
    sgt2.append(create_segments(i))
    
sgt_end2 =[item for lista in sgt2 for item in lista]

200
200
200
200
200
200
200
200
200
200


In [41]:
#unir offers con segments
for i in global_offer_csv.offer_name:
    for j,z in sgt_end:
        if z==i:
            global_offer_csv.loc[global_offer_csv.offer_name == i, ('customer_segment_id')] = j

In [42]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,20230701_BAQ_comidarapida_piloto_NB_aceitesygr...,comidarapida_piloto_NB_aceitesygrasas,6,true,01-07-2023,31-07-2023,7,1,181402,632389#624989#222614
1,20230701_BAQ_comidarapida_piloto_NB_arroz_W26,comidarapida_piloto_NB_arroz,4,true,01-07-2023,31-07-2023,7,1,181406,554034#281192
2,20230701_BAQ_comidarapida_piloto_NB_azucaryend...,comidarapida_piloto_NB_azucaryendulzantes,4,true,01-07-2023,31-07-2023,5,1,181407,83616#600127
3,20230701_BAQ_comidarapida_piloto_NB_harinasyme...,comidarapida_piloto_NB_harinasymezclas,4,true,01-07-2023,31-07-2023,6,1,181420,30861#257373#602333
4,20230701_BAQ_comidarapida_piloto_NB_pasta_W26,comidarapida_piloto_NB_pasta,12,true,01-07-2023,31-07-2023,0,1,181425,257365#257363


In [43]:
global_offer_csv = global_offer_csv.loc[~global_offer_csv.customer_segment_id.isnull()].reset_index(drop=True)
global_offer_csv.loc[global_offer_csv['offer_name'].str.contains("control", case=False), ('discount','max_uses_per_order')]=1
global_offer_csv.loc[global_offer_csv['offer_name'].str.contains("control", case=False), ('start_date','end_date')]='01-01-2028'
global_offer_csv.loc[global_offer_csv['offer_name'].str.contains("control", case=False), ('end_date')]='02-01-2028'
global_offer_csv.loc[global_offer_csv['offer_name'].str.contains("control", case=False), ('sku_id')]= global_offer_csv.loc[~global_offer_csv['offer_name'].str.contains("piloto", case=False), ('sku_id')].tolist()

In [44]:
len_offer =int(len(global_offer_csv)/4)
len_offer = [global_offer_csv[:len_seg],global_offer_csv[len_seg:len_seg*2],global_offer_csv[len_seg*2:len_seg*3],global_offer_csv[len_seg*3:]]

In [45]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,20230701_BAQ_comidarapida_piloto_NB_aceitesygr...,comidarapida_piloto_NB_aceitesygrasas,6,true,01-07-2023,31-07-2023,7,1,181402,632389#624989#222614
1,20230701_BAQ_comidarapida_piloto_NB_arroz_W26,comidarapida_piloto_NB_arroz,4,true,01-07-2023,31-07-2023,7,1,181406,554034#281192
2,20230701_BAQ_comidarapida_piloto_NB_azucaryend...,comidarapida_piloto_NB_azucaryendulzantes,4,true,01-07-2023,31-07-2023,5,1,181407,83616#600127
3,20230701_BAQ_comidarapida_piloto_NB_harinasyme...,comidarapida_piloto_NB_harinasymezclas,4,true,01-07-2023,31-07-2023,6,1,181420,30861#257373#602333
4,20230701_BAQ_comidarapida_piloto_NB_pasta_W26,comidarapida_piloto_NB_pasta,12,true,01-07-2023,31-07-2023,0,1,181425,257365#257363


In [46]:
#//--------------------------------------------
#//Exporting single file data
global_offer_csv.to_csv(f'SD/export_data/MS/csv_{city_code}_All_MS.csv', index = False)

for i,offer in enumerate(len_offer):
    offer.to_csv(f'SD/export_data/MS/csv_{city_code}_All_MS_{i}.csv', index = False)

with open(f'SD/export_data/MS/nb_{city_code}_All_MS.txt', 'w') as outfile:
        json.dump(global_segment_list, outfile)    
with open(f'SD/export_data/MS/nb_{city_code}_All_MS.json', 'w') as outfile:
        json.dump(global_segment_list, outfile)
print(f'Done {city_code}')

Done BAQ
